In [11]:
# Uncomment below if you want to run this file only
%run main.ipynb
#%run data_cleaning.ipynb
#%run data_visualization.ipynb
#%run feature_engineering.ipynb

In [7]:
# Read the CSV From FEATURE ENGINEERING data source file from S3 into a DataFrame
# Use the methods from the S3Utils class
if s3_utils.check_file_exists(output_file_key_data_feature_engineering):
    data = s3_utils.read_csv_from_s3(output_file_key_data_feature_engineering)

In [8]:
# Separate features and target
X = data.drop('target', axis=1)  # Replace 'target' with the name of your target column
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
# Create a pipeline for preprocessing and resampling
pipeline = ImbPipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
    ('scaler', StandardScaler()),                 # Scale features
    ('smote', SMOTE(random_state=42))             # Apply SMOTE
])

# Fit the pipeline on the training data and transform it
X_train_res, y_train_res = pipeline.fit_resample(X_train, y_train)


In [12]:

# Train an SVM classifier
model = SVC(kernel='linear', class_weight='balanced')
model.fit(X_train_res, y_train_res)

# Make predictions on the test set
predictions = model.predict(X_test)



In [ ]:

# Evaluate the model
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))


In [ ]:

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [1, 0.1, 0.01],
    'kernel': ['rbf', 'linear']
}

grid_search = GridSearchCV(svm.SVC(class_weight='balanced'), param_grid, refit=True, verbose=2)
grid_search.fit(X_train_res, y_train_res)

# Evaluate the best model found by grid search
best_model = grid_search.best_estimator_
best_predictions = best_model.predict(X_test)
print(classification_report(y_test, best_predictions))
